In [1]:
import tensorflow as tf
import pandas as pd
import csv
import joblib
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import os
from tensorflow.keras.models import load_model

In [3]:
# read a submitted csv file
data_dir = "/home/acapskyrim/projects/psm/psm_learn/data"

for filename in os.listdir(data_dir):
    owner = filename
    dataset = pd.read_csv(os.path.join(data_dir, filename), parse_dates=[0])
    dataset.columns = ['Time', 'Beacon_1', 'Beacon_2', 'Beacon_3', 'Beacon_4', 'Beacon_5']

dataset.set_index('Time', inplace=True)

In [7]:
# normalize data
scaler = joblib.load("./scaler_data")
threshold = joblib.load("./threshold")
x = scaler.transform(dataset)
x = x.reshape(x.shape[0], 1, x.shape[1])

In [28]:
data_out = {}

with tf.Graph().as_default():
    model = load_model("user1_model.h5")
    data_out["Analysis"] = []
#     model = define_model()
    preds = model.predict(x)
    preds = preds.reshape(preds.shape[0], preds.shape[2])
    preds = pd.DataFrame(preds, columns = dataset.columns)
    preds.index = dataset.index
    
    scored = pd.DataFrame(index=dataset.index)
    yhat = x.reshape(x.shape[0], x.shape[2])
    scored['Loss_mae'] = np.mean(np.abs(yhat - preds), axis=1)
    scored['Threshold'] = threshold
    scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']
    
    triggered = []
    for i in range(len(scored)):
        temp = scored.iloc[i]
        if temp.iloc[2]:
            triggered.append(temp)
    print(len(triggered))
    if len(triggered) > 0:
        for j in range(len(triggered)):
            out = triggered[j]
            result = {"Anomaly": True, "date": f"{out.name.year:02d}" + "-" + f"{out.name.month:02d}" + "-" + f"{out.name.day:02d}", "time": f"{out.name.hour:02d}" + ":" + f"{out.name.minute:02d}"}
            data_out["Analysis"].append(result)

2


In [29]:
data_out

{'Analysis': [{'Anomaly': True, 'date': '2020-01-08', 'time': '09:00'},
  {'Anomaly': True, 'date': '2020-01-08', 'time': '17:00'}]}